<a href="https://colab.research.google.com/github/sainikhit2k/Automatic-assigment-grading-using-LSTM-RNN-and-NN/blob/master/TwoLayerNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# from google.colab import files
# files.upload()


drive  kaggle.json  sample_data  training_set_rel3.tsv


In [0]:
!pip install -q kaggle


In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
!kaggle competitions download -c asap-aes

  0% 0.00/1.60M [00:00<?, ?B/s]
100% 1.60M/1.60M [00:00<00:00, 111MB/s]
100% 4.86M/4.86M [00:00<00:00, 5.17MB/s]

Training_Materials.zip: Skipping, found more recently modified local copy (use --force to force download)
  0% 0.00/214k [00:00<?, ?B/s]
100% 214k/214k [00:00<00:00, 65.7MB/s]
  0% 0.00/2.01M [00:00<?, ?B/s]
100% 2.01M/2.01M [00:00<00:00, 66.1MB/s]
  0% 0.00/88.5k [00:00<?, ?B/s]
100% 88.5k/88.5k [00:00<00:00, 85.3MB/s]
 81% 5.00M/6.20M [00:00<00:00, 7.50MB/s]
100% 6.20M/6.20M [00:00<00:00, 9.08MB/s]
  0% 0.00/41.4k [00:00<?, ?B/s]
100% 41.4k/41.4k [00:00<00:00, 37.5MB/s]
 91% 5.00M/5.49M [00:00<00:00, 6.31MB/s]
100% 5.49M/5.49M [00:00<00:00, 6.65MB/s]
  0% 0.00/14.9k [00:00<?, ?B/s]
100% 14.9k/14.9k [00:00<00:00, 13.4MB/s]
  0% 0.00/14.9k [00:00<?, ?B/s]
100% 14.9k/14.9k [00:00<00:00, 12.8MB/s]
  0% 0.00/1.79M [00:00<?, ?B/s]
100% 1.79M/1.79M [00:00<00:00, 121MB/s]
test_set.tsv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
!unzip training_set_rel3.tsv.zip

Archive:  training_set_rel3.tsv.zip
replace training_set_rel3.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
import pandas as pd

df = pd.read_csv("training_set_rel3.tsv", sep = "\t", encoding = "ISO-8859-1")

df = df[['essay', 'domain1_score']]

In [0]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec


def essay_to_wordlist(essay):
  essay = re.sub("[^a-zA-Z]", " ", essay)
  words = essay.strip().lower().split()
  stop_words = set(stopwords.words("english"))
  words = [w for w in words if w not in stop_words]
  return words


def essay_to_sentences(essay):
  tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
  raw_sentences = tokenizer.tokenize(essay.strip())
  sentences = []
  for raw_sentence in raw_sentences:
    if(len(raw_sentence) > 0):
      sentences.append(essay_to_wordlist(raw_sentence))
  return sentences

def makeFeatureVec(words, model, num_features):
  featureVec = np.zeros((num_features,))
  num_words = 0
  index2word = set(model.wv.index2word)
  for word in words:
    if word in index2word:
      featureVec = np.add(featureVec, model[word])
      num_words += 1
  return np.divide(featureVec, num_words)


def getFeatureVecs(essays, model, num_features):
  feature_matrix = np.zeros((len(essays), num_features), dtype = "float32")
  essay_count = 0
  for essay in essays:
    feature_matrix[essay_count] = makeFeatureVec(essay, model, num_features)
    essay_count += 1
  return feature_matrix


In [0]:
from keras.layers import Dense, Dropout, Conv1D, Flatten
from keras.models import Sequential

def get_nn_model(num_dims = 300):
  model = Sequential()
  model.add(Conv1D(filters = 1, kernel_size = 3, input_shape = (num_dims, 1), strides = 3, padding = "valid"))
  model.add(Flatten())
  model.add(Dense(300, activation = "relu"))
  model.add(Dropout(rate = 0.5))
  model.add(Dense(150, activation = "relu"))
  model.add(Dropout(rate = 0.5))
  model.add(Dense(1, activation = "relu"))
  model.compile(loss = 'mean_squared_error', optimizer = 'rmsprop', metrics = ['mae'])
  return model



In [0]:
X = df['essay']
Y = df['domain1_score']
print(len(X))

12976


In [0]:
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import KFold
import numpy as np
import pickle


def train_model(X, y, num_dims = 300, batch_size = 64, epochs = 50):
  cv = KFold(n_splits = 5, shuffle = True)

  # model_name = "nn_" + "dims:" + str(num_dims) + "batch:" + str(batch_size) + "epochs:" + str(epochs)
  # word2vec_name = model_name + ":word2vec_model"
  fold_count = 1

  best_hist = None
  best_score = 0.0
  best_model = None
  best_word_vec = None

  scores = []

  for train_ind, test_ind in cv.split(X):

    print("------------------Fold {}------------------".format(fold_count))
    X_train, X_test, Y_train, Y_test = X[train_ind], X[test_ind], Y[train_ind], Y[test_ind]

    sentences = []
    for essay in X_train:
      sentences += essay_to_sentences(essay)
    
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3

    model = Word2Vec(sentences, workers = num_workers, size = num_dims, sample = downsampling, window = context)
    print("Word2Vec model trained")
    
    train_essays = [essay_to_wordlist(essay) for essay in X_train]
    test_essays = [essay_to_wordlist(essay) for essay in X_test]

    train_essays = getFeatureVecs(train_essays, model, num_dims)

    test_essays = getFeatureVecs(test_essays, model, num_dims)
    
    train_essays = np.expand_dims(train_essays, axis = -1)
    test_essays = np.expand_dims(test_essays, axis = -1)

    nn_model = get_nn_model(num_dims=num_dims)
    # nn_model.summary()
    history = nn_model.fit(train_essays, Y_train.values, batch_size = batch_size, epochs = epochs)

    


    y_pred = nn_model.predict(test_essays)
    y_pred = np.round(y_pred)

    score = cohen_kappa_score(Y_test.values, y_pred, weights = "quadratic")
    print("Kappa score: {}".format(score))
    scores.append(score)

    if score > best_score:
      best_score = score
      best_hist = history
      best_model = nn_model
      best_word_vec = model


    fold_count += 1
  

  
  print("Best Score: {}".format(best_score))
  print("Average Kappa Score: {}".format(np.mean(scores)))
  return best_model, best_hist, best_score




In [0]:
model, history, score = train_model(X, Y, num_dims=300)

------------------Fold 1------------------
Word2Vec model trained


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


ValueError: ignored

In [0]:
import pickle
import json

def save_model(model, history_dict, name):
  model.save(name + ".h5")
  with open(name + ".p", 'wb') as fp:
    pickle.dump(history_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

save_model(model, history, "300_64_50_trained")

In [0]:
model, history, score = train_model(X, Y, num_dims=200)
save_model(model, history, "200_64_50_trained")

------------------Fold 1------------------
Word2Vec model trained


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


ValueError: ignored

In [0]:
model, history, score = train_model(X, Y, num_dims=100)
save_model(model, history, "100_64_50_trained")

------------------Fold 1------------------
Word2Vec model trained


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_27 (Conv1D)           (None, 33, 1)             4         
_________________________________________________________________
flatten_27 (Flatten)         (None, 33)                0         
_________________________________________________________________
dense_79 (Dense)             (None, 300)               10200     
_________________________________________________________________
dropout_53 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_80 (Dense)             (None, 150)               45150     
_________________________________________________________________
dropout_54 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_81 (Dense)             (None, 1)               

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_28 (Conv1D)           (None, 33, 1)             4         
_________________________________________________________________
flatten_28 (Flatten)         (None, 33)                0         
_________________________________________________________________
dense_82 (Dense)             (None, 300)               10200     
_________________________________________________________________
dropout_55 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_83 (Dense)             (None, 150)               45150     
_________________________________________________________________
dropout_56 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_84 (Dense)             (None, 1)               

...

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_29 (Conv1D)           (None, 33, 1)             4         
_________________________________________________________________
flatten_29 (Flatten)         (None, 33)                0         
_________________________________________________________________
dense_85 (Dense)             (None, 300)               10200     
_________________________________________________________________
dropout_57 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_86 (Dense)             (None, 150)               45150     
_________________________________________________________________
dropout_58 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_87 (Dense)             (None, 1)               

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_30 (Conv1D)           (None, 33, 1)             4         
_________________________________________________________________
flatten_30 (Flatten)         (None, 33)                0         
_________________________________________________________________
dense_88 (Dense)             (None, 300)               10200     
_________________________________________________________________
dropout_59 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_89 (Dense)             (None, 150)               45150     
_________________________________________________________________
dropout_60 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_90 (Dense)             (None, 1)               

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_31 (Conv1D)           (None, 33, 1)             4         
_________________________________________________________________
flatten_31 (Flatten)         (None, 33)                0         
_________________________________________________________________
dense_91 (Dense)             (None, 300)               10200     
_________________________________________________________________
dropout_61 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_92 (Dense)             (None, 150)               45150     
_________________________________________________________________
dropout_62 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_93 (Dense)             (None, 1)               

In [0]:
model, history, score = train_model(X, Y, num_dims=50)
save_model(model, history, "50_64_50_trained")

------------------Fold 1------------------
Word2Vec model trained


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_32 (Conv1D)           (None, 16, 1)             4         
_________________________________________________________________
flatten_32 (Flatten)         (None, 16)                0         
_________________________________________________________________
dense_94 (Dense)             (None, 300)               5100      
_________________________________________________________________
dropout_63 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_95 (Dense)             (None, 150)               45150     
_________________________________________________________________
dropout_64 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_96 (Dense)             (None, 1)               

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_33 (Conv1D)           (None, 16, 1)             4         
_________________________________________________________________
flatten_33 (Flatten)         (None, 16)                0         
_________________________________________________________________
dense_97 (Dense)             (None, 300)               5100      
_________________________________________________________________
dropout_65 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_98 (Dense)             (None, 150)               45150     
_________________________________________________________________
dropout_66 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_99 (Dense)             (None, 1)               

...

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_34 (Conv1D)           (None, 16, 1)             4         
_________________________________________________________________
flatten_34 (Flatten)         (None, 16)                0         
_________________________________________________________________
dense_100 (Dense)            (None, 300)               5100      
_________________________________________________________________
dropout_67 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_101 (Dense)            (None, 150)               45150     
_________________________________________________________________
dropout_68 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_102 (Dense)            (None, 1)               

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_35 (Conv1D)           (None, 16, 1)             4         
_________________________________________________________________
flatten_35 (Flatten)         (None, 16)                0         
_________________________________________________________________
dense_103 (Dense)            (None, 300)               5100      
_________________________________________________________________
dropout_69 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_104 (Dense)            (None, 150)               45150     
_________________________________________________________________
dropout_70 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_105 (Dense)            (None, 1)               

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_36 (Conv1D)           (None, 16, 1)             4         
_________________________________________________________________
flatten_36 (Flatten)         (None, 16)                0         
_________________________________________________________________
dense_106 (Dense)            (None, 300)               5100      
_________________________________________________________________
dropout_71 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_107 (Dense)            (None, 150)               45150     
_________________________________________________________________
dropout_72 (Dropout)         (None, 150)               0         
_________________________________________________________________
dense_108 (Dense)            (None, 1)               